## Web scrape unicornriot pages for all videos


In [ ]:
import requests
from bs4 import BeautifulSoup
import html5lib
import os, os.path
import pandas as pd



In [ ]:


OUT_ROOT_PATH = '/media/beanbagthomas/AED47533D474FEC1/Large_Storage/data/patriotfrontleak/'
original_link = 'http://vault.unicornriot.ninja/patriotfrontleaks/'


#change to True if you want to run download_visual_media function
download_media = False
# used if download_media is true - specifies extensions you want to download
filetype_to_download = [
    '.WEBM', '.MPG', '.MP2', '.MPEG', '.MPE', '.MPV', 
    '.OGG', '.MP4', '.M4P', '.M4V', '.AVI', '.WMV', 
    '.MOV', '.QT ', '.FLV', '.SWF' 
    ]


#change to True if you want to run create a csv with all files
create_a_csv = True


In [ ]:
def safe_open_wb(path):
    ''' Open "path" for writing, creating any parent directories as needed.
    '''
    os.makedirs(os.path.dirname(path), exist_ok=True)
    return open(path, 'wb')


def download_visual_media(full_link, file_name, ftd=filetype_to_download, out_path=OUT_ROOT_PATH):
    '''download images and video, extensions passed in ftd
    '''
    r = requests.get(full_link, stream=True)

    with safe_open_wb(OUT_ROOT_PATH + file_name) as f:
        print(f"downloading: {file_name}")
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk)


In [ ]:

"""Returns all scraped files with preserved file hierarchy

DESCRIPTION:
This program is custom designed for the unicornriot patriotfront leaks to parse
and download the vast dataset. Customizable by file extension.

TODO
Test on nonvideo, images, docs
"""

link = original_link
traversed = []
link_start = "http://vault.unicornriot.ninja/patriotfrontleaks/"

if create_a_csv:
    df = pd.DataFrame(columns=['file name', 'directory path', 'link'])

while True:
    
    req = requests.get(link)
    soup = BeautifulSoup(req.text, "html5lib")

    #print(link)
    query = soup.findAll('a')
    current_dir_contents = []

    # iterate through page and grab first link
    for a_tag in query:
        a_link = a_tag['href']
        
        full_link = link + a_link
        file_name = full_link.split(link_start)[1]

        #download media if in filetype_to_download
        if any(ext in a_link.upper() for ext in filetype_to_download):
            if download_media == True:
                download_visual_media(full_link=full_link, file_name=file_name)

        if a_link != '../':
            current_dir_contents.append(full_link)

            if create_a_csv and '/' not in a_link and '.' in a_link:
                data_entry = {'file name': a_link.replace('%', ' '), 
                    'directory path': file_name.replace('%', ' '), 'link': full_link}

                df = df.append(data_entry, ignore_index=True)

        # make sure its actual directory and has not been traversed yet, then traverse
        if '/' in a_link and full_link not in traversed and a_link != "../":  
            link += a_link
            break

    #make list of only links
    current_dir_links = [string for string in current_dir_contents if '/' in string[-1]]   

    #break loop if at original link and all current_dir_links in traversed
    if link == original_link and all(elem in traversed for elem in current_dir_links):
        break

    # if bottom of a dir is reached or all have already been traversed, add to traversed
    if len(current_dir_links) == 0 or all(elem in traversed for elem in current_dir_links):  
        traversed.append(link)
        link = link.rsplit('/', 2)[0] + '/'
        continue

df.to_csv(OUT_ROOT_PATH + "patriot_front_leak.csv", index_label='index')

# Format discord leaks

In [ ]:
import pandas as pd
import json
import datetime

In [ ]:
with open('SouthernFront.txt') as jsonfile:
    file = json.loads(jsonfile.read())


In [ ]:
users = {}
channels = {}

file.keys()

In [ ]:
userlist = file['meta']['users']
channellist = file['meta']['channels']

i = 0
for userid in userlist:
    users[i] = {'name': userlist[userid]['name'], 'id': userid, 'pointer': i}
    i += 1
for channelid in channellist:
    channels[channellist[channelid]['name']] = channelid


In [ ]:
users

In [ ]:
file['data'].keys()

In [ ]:
# iterate through each channel
for channel in file['data'].keys():
    full_channel = file['data'][channel]

    #iterate through each message
    for messageid in full_channel.keys():
        print(full_channel[messageid])

        #generate time
        timeepoch = full_channel[messageid]['t'] / 1000
        utc_time = datetime.datetime.utcfromtimestamp(timeepoch).strftime("%Y/%d/%m, %H:%M:%S")
        print(utc_time)

        #find user
        cur_user = users[full_channel[messageid]['u']]['name']
        
        
    break